## Applying Hierarchical LDA Model

Hierarchical Latent Dirichlet Allocation (hLDA) addresses the problem of learning topic hierarchies from data. The model relies on a non-parametric prior called the nested Chinese restaurant process, which allows for arbitrarily large branching factors and readily accommodates growing data collections. The hLDA model combines this prior with a likelihood that is based on a hierarchical variant of latent Dirichlet allocation [1].

Idea inspired from [Hierarchical Topic Models and the Nested Chinese Restaurant Process](http://www.cs.columbia.edu/~blei/papers/BleiGriffithsJordanTenenbaum2003.pdf)

**Reference:**

[1] David M. Blei , Michael I. Jordan , Thomas L. Griffiths , Joshua B. Tenenbaum, Hierarchical topic models and the nested chinese restaurant process, Proceedings of the 16th International Conference on Neural Information Processing Systems, p.17-24, December 09-11, 2003, Whistler, British Columbia, Canada

#### Hierarchical LDA Model example
<img src="https://i.loli.net/2018/09/21/5ba4c638ee55a.png" alt="Hierarchical_LDA_Model_example.png" title="Hierarchical_LDA_Model_example.png" width="60%" height="60%"/>

In [1]:
from gensim import corpora, models, similarities
from gensim.corpora.dictionary import Dictionary
from gensim.models.ldamodel import LdaModel
from gensim.models.ldamulticore import LdaMulticore

import pyLDAvis.gensim

import os
import pickle
import pandas as pd
import numpy as np
import warnings

pyLDAvis.enable_notebook()

#### 1. Generate Dictionary tokens-- In order to train LDA model later on
- params:
    - processed_doc_path: cleand preprocessed data folder path
    - non_german_file_path: a list of all English files
    - topic_related_documents: documents that belong to the same topic
    - document_limit: restrict the number of documents to process
- variables:
    - dictionary: a list of tokens loaded from processed files
    - filenames: a list of processed file names

In [2]:
def generate_dictionary_tokens(processed_doc_path, non_german_file_path, topic_related_documents, document_limit):
    # get a list of non_german files
    non_german_files = []
    with open(non_german_file_path, 'r') as fr:
        for line in fr:
            non_german_files.append(line.strip())

    dictionary_tokens = []
    filenames = []

    for root, dirs, files in os.walk(processed_doc_path):
        for f in sorted(files):
            if f in topic_related_documents:
                if f[:-11] not in non_german_files:
                    document_limit -= 1
                    try:
                        with open(root+'/'+f, 'rb') as fr:
                            filenames.append(f)
                            document_tokens = pickle.load(fr)
                            dictionary_tokens.append(document_tokens)
                    except:
                        print('Error while processing: ', f)

            if document_limit == 0:
                break

    return dictionary_tokens, filenames

#### 2. Apply the hierachical LDA Model -- In order to group those documents which seem to belong to the same topic
- params:
    - processed_doc_path: cleand preprocessed data folder
    - non_german_file_path: a list of all English files
    - topic_num: topics numbers to choose while training the model
    - document_limit: restrict the number of documents to process
- variables:
    - gensim_dictionary: [Dictionay](https://radimrehurek.com/gensim/corpora/dictionary.html) type, defined by gesim library 
    - corpus: doc2bow, tokenized and nomalized list
    - lda: LDA model after training with given dataset

In [3]:
def hierachicalLDA(processed_doc_path, non_german_file_path, topic_related_documents, topic_num=50, document_limit=1000):

    dictionary_tokens, filenames = generate_dictionary_tokens(
        processed_doc_path, non_german_file_path, topic_related_documents, document_limit)

    gensim_dictionary = Dictionary(dictionary_tokens)
    corpus = [gensim_dictionary.doc2bow(text) for text in dictionary_tokens]
    lda = LdaModel(corpus, num_topics=topic_num,
                   id2word=gensim_dictionary, iterations=200)
    topics = lda.show_topics(num_topics=-1, num_words=20)

    return lda, gensim_dictionary, corpus, filenames

### Demonstration:

**Steps**:
1. Load 200 preprocessed files to generate dictionary tokens list
2. Train initial LDA model with obtained dictionary tokens to generate desired topics, e.g. 15 topics derived from 200 files
3. Visualize initial LDA model with 15 topics
4. Classify documents to the same topic (by topic probability)
5. Apply hierachical LDA model based on grouped topic related documents (further generate 4 topics)
6. Visualize second LDA model with 4 topics

In [4]:
%%time
# relative path
non_german_file_path = './non_german_files.txt'
processed_doc_path = '../spacy_corpus/'
topic_num_level_1 = 15
# right now, treat every document to the same topic
topic_related_documents = os.listdir(processed_doc_path)
init_lda, init_gensim_dictionary, init_corpus, init_filenames = hierachicalLDA(
    processed_doc_path, non_german_file_path, topic_related_documents, topic_num_level_1, 200)

CPU times: user 24.5 s, sys: 228 ms, total: 24.8 s
Wall time: 8.45 s


#### Visualize Initial LDA model with 15 topics

In [5]:
pyLDAvis.gensim.prepare(init_lda, init_corpus, init_gensim_dictionary)

/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
10     0.008868  0.010302       1        1  25.606091
2     -0.012626 -0.000370       2        1  15.169480
5     -0.029363  0.011110       3        1  13.951346
13    -0.005817 -0.006579       4        1  12.556746
1      0.007123 -0.002770       5        1   8.689456
6     -0.001950  0.000524       6        1   5.295954
4     -0.012066 -0.012149       7        1   5.168394
3      0.001104 -0.012525       8        1   4.450998
0      0.014149  0.003489       9        1   2.692009
11     0.007794  0.003013      10        1   2.635686
8      0.009052  0.004799      11        1   2.103605
12     0.002259  0.006669      12        1   1.100343
7      0.002891 -0.006862      13        1   0.369282
9      0.007578 -0.005013      14        1   0.198153
14     0.001003  0.006362      15        1   0.012456, topic_info=      Category         Freq            Term        Total  loglift  logprob
term                                                                      
551    Default  8027.000000            höhe  8027.000000  30.0000  30.0000
292    Default  7851.000000         deutsch  7851.000000  29.0000  29.0000
1172   Default  6581.000000      wesentlich  6581.000000  28.0000  28.0000
3503   Default  6931.000000         vorjahr  6931.000000  27.0000  27.0000
634    Default  4386.000000         konzern  4386.000000  26.0000  26.0000
1060   Default  5616.000000     unternehmen  5616.000000  25.0000  25.0000
544    Default  5700.000000            hoch  5700.000000  24.0000  24.0000
35150  Default  4851.000000         daimler  4851.000000  23.0000  23.0000
294    Default  4985.000000        dezember  4985.000000  22.0000  22.0000
690    Default  4645.000000          liegen  4645.000000  21.0000  21.0000
904    Default  4725.000000         risiken  4725.000000  20.0000  20.0000
955    Default  4100.000000         sonstig  4100.000000  19.0000  19.0000
122    Default  3338.000000            bank  3338.000000  18.0000  18.0000
875    Default  3908.000000          rahmen  3908.000000  17.0000  17.0000
2430   Default  3284.000000        vorstand  3284.000000  16.0000  16.0000
1104   Default  4640.000000  vermögenswerte  4640.000000  15.0000  15.0000
26     Default  3423.000000          aktien  3423.000000  14.0000  14.0000
97     Default  4364.000000        aufgrund  4364.000000  13.0000  13.0000
195    Default  4056.000000        betragen  4056.000000  12.0000  12.0000
1378   Default  4019.000000        ergebnis  4019.000000  11.0000  11.0000
868    Default  4660.000000         quartal  4660.000000  10.0000  10.0000
573    Default  3399.000000    insbesondere  3399.000000   9.0000   9.0000
489    Default  3344.000000    gesellschaft  3344.000000   8.0000   8.0000
338    Default  3842.000000     entsprechen  3842.000000   7.0000   7.0000
1750   Default  3785.000000    aufsichtsrat  3785.000000   6.0000   6.0000
341    Default  4003.000000     entwicklung  4003.000000   5.0000   5.0000
348    Default  3376.000000        erfolgen  3376.000000   4.0000   4.0000
482    Default  3851.000000   geschäftsjahr  3851.000000   3.0000   3.0000
358    Default  3853.000000         erhöhen  3853.000000   2.0000   2.0000
574    Default  4000.000000       insgesamt  4000.000000   1.0000   1.0000
...        ...          ...             ...          ...      ...      ...
31843  Topic15     0.022935            büll    54.486748   1.2177  -9.1822
35150  Topic15     0.654459         daimler  4851.359375   0.0798  -5.8310
573    Topic15     0.488878    insbesondere  3399.490723   0.1437  -6.1227
1244   Topic15     0.297538      zusätzlich  1739.824341   0.3170  -6.6193
375    Topic15     0.355869         erträge  2219.471924   0.2525  -6.4403
1378   Topic15     0.545455        ergebnis  4019.652588   0.0857  -6.0132
264    Topic15     0.249324       corporate  1376.865601   0.3742  -6.7961
376    Topic15     0.481595        erwa

#### Group topic related documents by assuming that belong to the same topic with the probability larget than 0.5
- params: 
    - df: pandas DataFrame, each column represents one topic, and each row indicates the probability of that document belongs to this topic
- variables: 
    - topic_related_documents: list, contain group of doucment names which seem to belong to the same topic

In [6]:
def get_topic_related_doc(lda, corpus, filenames, topic_num):  
    
    doc_pos = 0
    mat = np.zeros((len(filenames), topic_num))

    for doc in corpus:
        vector = lda[doc] # get topic probability distribution for a document
        for element in vector:
            mat[doc_pos][element[0]] = element[1]
        doc_pos += 1

    df = pd.DataFrame(mat, index=filenames, columns=range(0,topic_num))

    topic_related_documents = []
    
    for column in range(df.shape[1]):
        row_count = 0
        for row in range(df.shape[0]):
            if df.iat[row, column] > 0.5:
                row_count += 1
        topic_related_documents.append(df.nlargest(row_count, column).index)
    
    return topic_related_documents



In [7]:
topic_related_documents = get_topic_related_doc(init_lda, init_corpus, init_filenames, topic_num_level_1)
df_topics = pd.DataFrame(topic_related_documents).transpose()
df_topics = df_topics.rename(columns=lambda x: 'Topic '+ str(x))

**Desigante documents to topics (each topic consists of a list of documnents)**

In [8]:
df_topics

,Topic 0,Topic 1,Topic 2,Topic 3,Topic 4,Topic 5,Topic 6,Topic 7,Topic 8,Topic 9,Topic 10,Topic 11,Topic 12,Topic 13,Topic 14
0,ADVA_Optical-QuarterlyReport-2012-Q3,DeutscheWohnen-AnnualReport-2011,FuchsPetrolub-AnnualReport-2011,BMW-QuarterlyReport-2014-Q3,DIC-Asset-AnnualReport-2013,Daimler-AnnualReport-2015,Diebold_Nixdorf-QuarterlyReport-2012-Q3,None,Cancom-AnnualReport-2011,Bilfinger-QuarterlyReport-2010-Q1,DeutscheBank-AnnualReport-2014,EVOTEC-QuarterlyReport-2012-Q2,BVB-QuarterlyReport-2011-Q1,Biotest-AnnualReport-2016,None
1,ADVA_Optical-QuarterlyReport-2016-Q3,DeutscheWohnen-AnnualReport-2010,Allianz-QuarterlyReport-2015-Q1,EnBW-QuarterlyReport-2011-Q3,DialogSemiconductor-QuarterlyReport-2015-Q1,Daimler-AnnualReport-2014,Diebold_Nixdorf-QuarterlyReport-2012-Q2,None,Cancom-AnnualReport-2010,BayerischeLandesbank-QuarterlyReport-2017-Q1,DeutscheBank-AnnualReport-2012,Elring_Klinger-QuarterlyReport-2013-Q3,EVOTEC-QuarterlyReport-2016-Q2,Biotest-AnnualReport-2011,None
2,Brenntag-AnnualReport-2015,Deutsche_Euroshop-QuarterlyReport-2010-Q2,Bayer-QuarterlyReport-2014-Q3,EnBW-QuarterlyReport-2011-Q2,Drillisch-QuarterlyReport-2013-Q1,Daimler-AnnualReport-2013,Bilfinger-QuarterlyReport-2014-Q1,None,Cancom-QuarterlyReport-2015-Q2,BetAtHome-QuarterlyReport-2017-Q1,DeutscheBank-AnnualReport-2013,DialogSemiconductor-QuarterlyReport-2011-Q1,ADVA_Optical-QuarterlyReport-2012-Q2,Biotest-AnnualReport-2010,None
3,Commerzbank-QuarterlyReport-2014-Q2,Deutsche_Euroshop-QuarterlyReport-2010-Q3,Fresenius-QuarterlyReport-2013-Q1,DIC-Asset-QuarterlyReport-2014-Q3,Drillisch-QuarterlyReport-2017-Q1,Daimler-AnnualReport-2012,Bertrandt-QuarterlyReport-2014-Q2,None,Cancom-QuarterlyReport-2015-Q3,None,DeutscheBank-AnnualReport-2015,Elring_Klinger-QuarterlyReport-2013-Q2,Diebold_Nixdorf-QuarterlyReport-2016-Q3,Deutsche_Euroshop-AnnualReport-2015,None
4,Beiersdorf-QuarterlyReport-2014-Q1,DeutscheWohnen-AnnualReport-2016,Bayer-QuarterlyReport-2014-Q2,CarlZeissMeditec-QuarterlyReport-2012-Q1,DeutscheBoerse-QuarterlyReport-2017-Q1,AmadeusFiRe-AnnualReport-2016,EvonikIndustries-QuarterlyReport-2015-Q2,None,Cancom-QuarterlyReport-2011-Q2,None,DeutscheBank-QuarterlyReport-2013-Q3,Aareal-QuarterlyReport-2013-Q1,None,Deutsche_Euroshop-AnnualReport-2014,None
5,ADVA_Optical-QuarterlyReport-2016-Q2,Continental-AnnualReport-2012,Durr-QuarterlyReport-2013-Q3,Fraport-QuarterlyReport-2012-Q1,Freenet-QuarterlyReport-2011-Q1,AmadeusFiRe-AnnualReport-2010,Bertrandt-QuarterlyReport-2014-Q3,None,Cancom-QuarterlyReport-2011-Q3,None,DeutscheBank-QuarterlyReport-2013-Q2,Commerzbank-QuarterlyReport-2010-Q2,None,Fresenius_Medical_Care-QuarterlyReport-2016-Q3,None
6,Commerzbank-QuarterlyReport-2014-Q3,Deutsche_Euroshop-QuarterlyReport-2014-Q2,Capital_Stage-QuarterlyReport-2013-Q3,Fielmann-QuarterlyReport-2012-Q1,DIC-Asset-AnnualReport-2014,Daimler-QuarterlyReport-2010-Q3,eon-AnnualReport-2010,None,EvonikIndustries-QuarterlyReport-2011-Q3,None,Deutsche_Post-AnnualReport-2015,Elring_Klinger-QuarterlyReport-2017-Q2,None,Fresenius_Medical_Care-QuarterlyReport-2016-Q2,None
7,None,DeutscheWohnen-QuarterlyReport-2017-Q1,Fresenius-QuarterlyReport-2017-Q1,DeutscheWohnen-QuarterlyReport-2013-Q1,DeutscheBoerse-QuarterlyReport-2013-Q1,Daimler-QuarterlyReport-2010-Q2,DIC-Asset-QuarterlyReport-2014-Q2,None,None,None,Deutsche_Post-AnnualReport-2014,Beiersdorf-QuarterlyReport-2010-Q1,None,EnBW-QuarterlyReport-2015-Q2,None
8,None,Deutsche_Euroshop-AnnualReport-2012,Bayer-QuarterlyReport-2010-Q2,Bechtle-QuarterlyReport-2011-Q3,DIC-Asset-AnnualReport-2015,Daimler-QuarterlyReport-2014-Q3,None,None,None,None,DzBank-QuarterlyReport-2015-Q1,Aareal-QuarterlyReport-2017-Q1,None,Eventim-AnnualReport-2015,None
9,None,Continental-QuarterlyReport-2013-Q2,Bayer-QuarterlyReport-2010-Q3,eon-QuarterlyReport-2015-Q1,CompuGroupMedical-QuarterlyReport-2010-Q1,Daimler-QuarterlyReport-2014-Q2,None,None,None,None,Deutsche_Post-AnnualReport-2012,Commerzbank-QuarterlyReport-2010-Q3,None,Eventim-AnnualReport-2014,None


#### Give a simple example, apply the Hierarchical LDA on one topic (e.g Topic 2) that has the maximum related documents to further generate 4 topics

In [9]:
topic_num_level_2 = 4
topic_docs = max(topic_related_documents, key=len)
lda, gensim_dictionary, corpus, filenames = hierachicalLDA(
    processed_doc_path, non_german_file_path, topic_docs.tolist(), topic_num_level_2)
pyLDAvis.gensim.prepare(lda, corpus, gensim_dictionary)

/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.012632  0.000441       1        1  36.652615
3     -0.014789 -0.002991       2        1  25.394693
1      0.004016 -0.007672       3        1  24.049292
2     -0.001858  0.010221       4        1  13.903399, topic_info=      Category         Freq            Term        Total  loglift  logprob
term                                                                      
2249   Default   343.000000       millionen   343.000000  30.0000  30.0000
404    Default  1254.000000            höhe  1254.000000  29.0000  29.0000
400    Default  1019.000000            hoch  1019.000000  28.0000  28.0000
20392  Default   288.000000       fresenius   288.000000  27.0000  27.0000
257    Default   883.000000        ergebnis   883.000000  26.0000  26.0000
860    Default  1278.000000         vorjahr  1278.000000  25.0000  25.0000
406    Default   538.000000            ifrs   538.000000  24.0000  24.0000
64     Default   646.000000        aufgrund   646.000000  23.0000  23.0000
16     Default    78.000000         aixtron    78.000000  22.0000  22.0000
718    Default   720.000000         steigen   720.000000  21.0000  21.0000
241    Default   573.000000       enthalten   573.000000  20.0000  20.0000
822    Default   577.000000  vermögenswerte   577.000000  19.0000  19.0000
242    Default   560.000000     entsprechen   560.000000  18.0000  18.0000
508    Default   807.000000          liegen   807.000000  17.0000  17.0000
419    Default   571.000000       insgesamt   571.000000  16.0000  16.0000
650    Default   571.000000         risiken   571.000000  15.0000  15.0000
49     Default   444.000000       ansteigen   444.000000  14.0000  14.0000
615    Default   513.000000         prozent   513.000000  13.0000  13.0000
772    Default   768.000000          umsatz   768.000000  12.0000  12.0000
254    Default   512.000000        erfolgen   512.000000  11.0000  11.0000
1723   Default   478.000000     deutschland   478.000000  10.0000  10.0000
21542  Default   677.000000           fuchs   677.000000   9.0000   9.0000
1429   Default   217.000000         allianz   217.000000   8.0000   8.0000
787    Default   858.000000     unternehmen   858.000000   7.0000   7.0000
623    Default  1124.000000         quartal  1124.000000   6.0000   6.0000
874    Default   347.000000        vorstand   347.000000   5.0000   5.0000
2248   Default    90.000000      milliarden    90.000000   4.0000   4.0000
278    Default   592.000000        erwarten   592.000000   3.0000   3.0000
8519   Default   241.000000           bayer   241.000000   2.0000   2.0000
576    Default   268.000000         niedrig   268.000000   1.0000   1.0000
...        ...          ...             ...          ...      ...      ...
406     Topic4    93.198425            ifrs   538.653931   0.2187  -5.9816
615     Topic4    89.544502         prozent   513.816406   0.2259  -6.0216
419     Topic4    96.357231       insgesamt   571.517334   0.1928  -5.9482
400     Topic4   153.809708            hoch  1019.343506   0.0818  -5.4806
1420    Topic4    61.909710          aktien   335.398254   0.2834  -6.3906
874     Topic4    63.608574        vorstand   347.105927   0.2762  -6.3636
254     Topic4    86.433189        erfolgen   512.356201   0.1934  -6.0569
718     Topic4   110.928818         steigen   720.611084   0.1018  -5.8074
822     Topic4    92.798851  vermögenswerte   577.450317   0.1448  -5.9859
278     Topic4    93.987900        erwarten   592.810059   0.1313  -5.9731
1722    Topic4    71.444771         deutsch   411.332916   0.2226  -6.2474
1236    Topic4    81.110359          rahmen   489.653076   0.1752  -6.1205
588     Topic4    70.691643        operativ   418.883179   0.1938  -6.2580
246     Topic4   102.119568     entwicklung   721.324402   0.0181  -5.8902
1987    Topic4    51.232937          global   268.290009   0.3174  -6.5799
193     Topic4    85.860245      

First, we could apply initial LDA model to generalize topics from the whole documents, in the above case, we derive 15 topics from 200 files. Then, we designate each document to certain topic with high topic probability. 

Further, for demonstration, we choose the topic with maximum related documents to apply LDA model again to derive 4 topics. In this way, we assume it is possible to derive more detailed topics based on the previously generated topics. 

To sum up, it is possible to apply LDA model hierarchically to obtain better results. 